# Préparation des données pour l'analyse

Ce notebook a pour objectif de centraliser et préparer les données nécessaires pour l'étude de marché.

## 1. Chargement des bibliothèques

In [1]:
# Monter Google Drive pour accéder aux fichiers
import pandas as pd
import numpy as np
from google.colab import drive
from geopy.distance import geodesic

# Monter Google Drive
drive.mount('/content/drive')
path_drive = '/content/drive/My Drive/Projet_10/data/'

Mounted at /content/drive


## 2. Chargement des données brutes

In [2]:
# Chargement des fichiers CSV contenant les données brutes
# Lecture des fichiers CSV
df_dispo = pd.read_csv(f'{path_drive}DisponibiliteAlimentaire_2017.csv', sep=",")
df_pib = pd.read_csv(f'{path_drive}PIB_2017.csv', sep=",")
df_population = pd.read_csv(f'{path_drive}Population_2000_2018.csv', sep=",")
df_political = pd.read_csv(f'{path_drive}political_stability.csv', sep=",")
df_cordonnees = pd.read_csv(f'{path_drive}pays_coordonnees.csv', sep=",")

## 3. Vérification des doublons

In [3]:
# Vérification de la présence de doublons dans les différentes sources
print(f"Nombre de doublons dispo alimentaire : {df_dispo.duplicated().sum()}")
print(f"Nombre de doublons PIB : {df_pib.duplicated().sum()}")
print(f"Nombre de doublons population : {df_population.duplicated().sum()}")
print(f"Nombre de doublons stabilité politique : {df_political.duplicated().sum()}")
print(f"Nombre de doublons coordonnées : {df_cordonnees.duplicated().sum()}")

Nombre de doublons dispo alimentaire : 0
Nombre de doublons PIB : 0
Nombre de doublons population : 0
Nombre de doublons stabilité politique : 0
Nombre de doublons coordonnées : 0


## 4. Sélection et nettoyage des colonnes

In [4]:
# Filtrer les données PIB pour ne garder que l'année 2017 et les valeurs par habitant
df_pib = df_pib[(df_pib["Code année"] == 2017) & (df_pib["Élément"] == "Valeur US $ par habitant")]
df_pib = df_pib[["Zone", "Valeur"]].rename(columns={"Valeur": "PIB_habitant"})

In [5]:
# Filtrer les données de population pour l'année 2012
df_population_2012 = df_population[df_population["Code année"] == 2012]
df_population_2012 = df_population_2012[["Zone", "Valeur"]].rename(columns={"Valeur": "population_2012"})

# Filtrer les données de population pour l'année 2017
df_population_2017 = df_population[df_population["Code année"] == 2017]
df_population_2017 = df_population_2017[["Zone", "Valeur"]].rename(columns={"Valeur": "population"})

df_population = pd.merge(df_population_2017, df_population_2012, on="Zone")

# Changer l'unité de population
df_population['population'] = df_population['population'] * 1000
df_population['population_2012'] = df_population['population_2012'] * 1000

# Calcul la croissance de la population entre 2012 et 2017
df_population["croissance_population_5ans"] = ((df_population["population"] - df_population["population_2012"]) / df_population["population_2012"]) * 100

df_population.drop(columns=["population_2012"], inplace=True)

In [6]:
df_population

,Zone,population,croissance_population_5ans
0,Afghanistan,36296113.0,16.477889
1,Afrique du Sud,57009756.0,7.906280
2,Albanie,2884169.0,-1.026974
3,Algérie,41389189.0,10.713956
4,Allemagne,82658409.0,2.081915
...,...,...,...
231,Venezuela (République bolivarienne du),29402484.0,0.141845
232,Viet Nam,94600648.0,5.343674
233,Yémen,27834819.0,13.736022
234,Zambie,16853599.0,16.511747


In [7]:
# Filtrer les données de stabilité politique pour 2017
df_political = df_political[df_political["Code année"] == 2017]
df_political = df_political[["Zone", "Valeur"]].rename(columns={"Valeur": "stabilite_politique"})

In [8]:
# Filtrer les données alimentaires pour ne garder que les produits de type Volaille
df_dispo = df_dispo[df_dispo["Produit"].str.contains("Volaille", na=False)]
df_dispo = df_dispo.pivot(index="Zone", columns="Élément", values="Valeur").reset_index()

In [9]:
# Sélection des colonnes pertinentes pour l'analyse
elements_to_keep = [
    "Zone",
    "Disponibilité alimentaire en quantité (kg/personne/an)",
    "Disponibilité de protéines en quantité (g/personne/jour)",
    "Production",
    "Importations - Quantité",
    "Disponibilité intérieure",
]
df_dispo = df_dispo[elements_to_keep]

In [10]:
# Calculer la distance avec Paris
df_cordonnees_paris = df_cordonnees[df_cordonnees["Pays"] == "France"]
paris_cordonnees = (df_cordonnees_paris["Latitude"].values[0], df_cordonnees_paris["Longitude"].values[0])

In [11]:
# Fonction pour calculer la distance entre Paris et la capitale de chaque pays
def calculate_distance(row):
    capitale_cordonnees = (row['Latitude'], row['Longitude'])
    return geodesic(paris_cordonnees, capitale_cordonnees).kilometers

df_cordonnees['distance_paris_km'] = df_cordonnees.apply(calculate_distance, axis=1)

In [12]:
df_cordonnees = df_cordonnees.rename(columns={"Pays": "Zone"})
df_cordonnees.drop(columns=["Latitude", "Longitude"], inplace=True)

## 5. Merge des données

In [13]:
# Fusion de toutes les sources de données dans un dataframe final
df_final = df_dispo.merge(df_population, on="Zone", how="left")
df_final = df_final.merge(df_pib, on="Zone", how="left")
df_final = df_final.merge(df_political, on="Zone", how="left")
df_final = df_final.merge(df_cordonnees, on="Zone", how="left")

## 6. Nettoyage et formatage final des données

In [14]:
# Renommer les colonnes pour des noms plus explicites
df_final = df_final.rename(columns={
    "Zone": "pays",
    "Disponibilité alimentaire en quantité (kg/personne/an)": "dispo_alimentaire",
    "Disponibilité de protéines en quantité (g/personne/jour)": "dispo_proteine",
    "Production": "production",
    "Importations - Quantité": "importations",
    "Disponibilité intérieure": "dispo_interieure"
})

In [15]:
# Calcul du PIB total par pays = PIB par habitant × population
df_final["PIB_total"] = df_final["PIB_habitant"] * df_final["population"]
df_final["PIB_total"] = df_final["PIB_total"].apply(lambda x: "{:.0f}".format(x))
df_final["population"] = df_final["population"].apply(lambda x: "{:.0f}".format(x))

In [16]:
# Vérification des valeurs manquantes
df_final.isnull().sum()

,0
pays,0
dispo_alimentaire,0
dispo_proteine,0
production,4
importations,2
dispo_interieure,2
population,0
croissance_population_5ans,0
PIB_habitant,2
stabilite_politique,5


In [17]:
df_final[df_final.isnull().any(axis=1)]

,pays,dispo_alimentaire,dispo_proteine,production,importations,dispo_interieure,population,croissance_population_5ans,PIB_habitant,stabilite_politique,distance_paris_km,PIB_total
33,"Chine, Taiwan Province de",33.17,11.01,652.0,161.0,785.0,23674546,1.403399,NaN,NaN,9849.822848,nan
34,"Chine, continentale",12.33,3.96,18236.0,452.0,18161.0,1421021791,2.659675,8729.136930,NaN,8238.778386,12404293794153
43,Djibouti,2.68,0.92,NaN,3.0,3.0,944099,8.750001,2659.854160,-0.73,5585.328142,2511165653
92,Maldives,13.50,4.70,NaN,12.0,12.0,496402,24.963687,10194.746472,0.22,8286.875773,5060692538
108,Nouvelle-Calédonie,38.71,14.05,1.0,9.0,11.0,277150,6.329104,32260.577425,NaN,16740.083932,8941019033
113,Ouzbékistan,1.96,0.63,NaN,NaN,NaN,31959785,8.524353,1943.340034,-0.24,5160.801448,62108729669
117,Pays-Bas,20.33,8.48,1100.0,608.0,372.0,17021347,1.366777,NaN,NaN,430.243200,nan
120,Polynésie française,47.40,16.20,1.0,15.0,15.0,276102,2.640912,19743.958993,NaN,15712.998986,5451346566
130,République démocratique populaire lao,10.91,3.59,NaN,NaN,NaN,6953035,7.890490,2439.462412,0.39,9249.861573,16961667532
134,Saint-Vincent-et-les Grenadines,72.31,25.10,0.0,9.0,8.0,109827,1.283718,7996.654529,0.87,NaN,878248577


In [18]:
# Ajout manuel de la valeur manquante pour la Chine
df_final.loc[df_final["pays"] == "Chine, continentale", "stabilite_politique"] = -0.23

In [19]:
df_final.dropna(inplace=True)

In [20]:
# Sauvegarde du dataframe final nettoyé dans un fichier CSV
df_final.to_csv(f'{path_drive}df_final.csv', index=False)